In [30]:
from prediction import jwt_access

url = "https://server.ecosystem.ai:3001/api"
username = "user@ecosystem.ai"
password = "12345"
auth = jwt_access.Authenticate(url, username, password)

Login Successful.


In [32]:
import nltk
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from prediction import nlp_utils
from prediction import prediction_utils
from prediction.apis import data_management_engine

In [35]:
df_events = pd.read_csv("data/events.csv")

In [36]:
df_events.head(10)

,EntryID,Language,Text,attachmentId,callId,channel,confidence,endTime,found,iteration,resourceID,resourceName,segmentId,startTime,subType,term,type
0,39,16,how much is my bill,0,1001,0,57,31.35,0,2,1103,Balance or Usage,-1,30.90,1,PSUT_30,1
1,40,16,talk three fifty,0,1001,0,53,280.93,0,2,1124,Tariffs Plans,-1,276.78,1,PSUT_25,1
2,41,16,what is your address,0,1001,0,54,292.89,0,2,1120,Account Authentication,-1,290.83,1,PSUT_2,1
3,42,16,who finances your vehicle,0,1001,0,42,306.02,0,2,1120,Account Authentication,-1,305.10,1,PSUT_44,1
4,43,16,the conversion takes a month,0,1001,0,29,339.77,0,2,1122,Migration,-1,338.83,1,PSUT_2,1
5,71,16,vodacom for you,0,1007,0,28,22.91,0,2,1124,Tariffs Plans,-1,22.36,1,PSUT_30,1
6,72,16,i am not happy with,0,1007,0,57,33.51,1,2,1131,Dissatisfaction,-1,33.09,1,PSUT_0,1
7,73,16,my account was migrated without,0,1007,0,10,50.37,0,2,1134,Fraud,-1,49.61,1,PSUT_8,1
8,74,16,please can you give me,0,1007,0,44,54.81,0,2,1120,Account Authentication,-1,54.14,1,PSUT_13,1
9,75,16,what is your id number,0,1007,0,68,69.07,1,2,1120,Account Authentication,-1,67.73,1,PSUT_1,1


In [37]:
df_meta = pd.read_csv("data/meta.csv")

In [88]:
df_meta.head(10)

,audioStatus,callDuration,callId,callTime,customerGroupId,customerId,externalId,programId
0,0,277.200012,13824,1520244962,[NONE],[NONE],2018-03-05 12.16.02 - 00.04.37 - 8230 - 082377...,1003
1,6,2133.386963,13825,1520245075,[NONE],[NONE],2018-03-05 12.17.55 - 00.35.33 - 0791109870 - ...,1003
2,0,78.680000,13826,1520245280,[NONE],[NONE],2018-03-05 12.21.20 - 00.01.19 - 100201478 - 4...,1003
3,0,45.959999,13827,1519873458,[NONE],[NONE],2018-03-01 05.04.18 - 00.00.45 - 0829973864 - ...,1003
4,0,397.440002,13828,1519876132,[NONE],[NONE],2018-03-01 05.48.52 - 00.06.37 - 0711284392 - ...,1003
5,0,361.239990,13829,1519876644,[NONE],[NONE],2018-03-01 05.57.24 - 00.06.01 - 0711284392 - ...,1003
6,0,46.880001,13830,1519876989,[NONE],[NONE],2018-03-01 06.03.09 - 00.00.47 - 8223 - 082780...,1003
7,0,158.559998,13831,1519877103,[NONE],[NONE],2018-03-01 06.05.03 - 00.02.38 - 4943 - 082991...,1003
8,0,579.039978,13832,1519880245,[NONE],[NONE],2018-03-01 06.57.25 - 00.09.39 - 0825806229 - ...,1003
9,2,3.680000,13833,1519880825,[NONE],[NONE],2018-03-01 07.07.05 - 00.00.03 - 8029 - 082130...,1003


In [39]:
df_metaex = pd.read_csv("data/metaex.csv")

In [93]:
df_metaex.head(50)

,callId,fieldName,fieldValue
0,1001,category,test2
1,1002,category,prepaid
2,1003,category,prepaid
3,1004,category,prepaid
4,1005,category,prepaid
5,1006,category,prepaid
6,1007,category,prepaid
7,1008,category,prepaid
8,1009,category,prepaid
9,1010,category,prepaid


In [84]:
data = {
    "callId": [],
    "text": [],
    "target": [],
    "callDuration": [],
    "lang_pref": []
}
count = 0
target_type = "Balance or Usage"
event_ids = set(df_events.callId.unique())
meta_ids = set(df_meta.callId.unique())
metaex_ids = set(df_metaex.callId.unique())
all_ids = list(event_ids.intersection(meta_ids).intersection(metaex_ids))
for cid in all_ids:
    count += 1
    if count >= 10:
        break
#     events
    output = df_events.loc[df_events["callId"] == cid]
    appended_text = " ".join(output["Text"])
    rn = output["resourceName"]
    target = 0
    if target_type in list(rn):
        target = 1
    data["callId"].append(cid)
    data["text"].append(appended_text)
    data["target"].append(target)
#     meta
    output = df_meta.loc[df_meta["callId"] == cid]
    call_duration = float(output["callDuration"])
    data["callDuration"].append(call_duration)
#     metaex
    output = df_metaex.loc[df_metaex["callId"] == cid]
    lang_output = output.loc[output["fieldName"] == "langpref"]
    lang_pref = lang_output["fieldValue"]
    if len(lang_pref) > 0:
        lang_pref = lang_pref.iloc[0]
    else:
        lang_pref = ""
    data["lang_pref"].append(lang_pref)
new_df = pd.DataFrame(data)

In [85]:
new_df.head(20)

,callId,text,target,callDuration,lang_pref
0,13824,vodacom for you vodacom for you i didn't get m...,1,277.200012,
1,13826,vodacom for you do you want me to block,0,78.680000,eng
2,13827,what is your address vodacom for you,0,45.959999,eng
3,13829,getting message to rica your account is paid v...,0,361.239990,eng
4,13830,vodacom for you what is your address,0,46.880001,
5,13831,taking money on my phone top up three fifteen ...,1,158.559998,
6,13832,can you confirm your last month's balance ther...,1,579.039978,eng
7,13835,when can i cancel vodacom for you how much is ...,1,143.639999,eng
8,13837,cell c what type of account what is content se...,1,1159.239990,eng


In [87]:
text_field = "text"
text_only = nlp_utils.clean(new_df, text_field)
text_only = nlp_utils.remove_non_english(text_only, text_field)
dropped_stop_text = nlp_utils.drop_stop_df(text_only, text_field)
stemmed_text = nlp_utils.stem_df(dropped_stop_text, text_field, "english")

stemmed_text.head(5)


,callId,text,target,callDuration,lang_pref
0,13824,didnt get invoic id due upgrad dont talk three...,1,277.200012,
1,13826,want block,0,78.680000,eng
2,13827,address,0,45.959999,eng
3,13829,get messag account boss cancel boss get manag ...,0,361.239990,eng
4,13830,address,0,46.880001,


In [10]:
from prediction.apis import data_management_engine

result = data_management_engine.add_document_database(auth, "nlp_testcase")
result = data_management_engine.add_document_collection(auth, "nlp_testcase", "spamtexts")

get /addMongoDatabase?database=nlp_testcase&
get /addMongoCollection?database=nlp_testcase&collection=spamtexts&


In [ ]:
print("Starting Training")
text_field = "text"
train_df = stemmed_text.loc[~stemmed_text["target"].isna()]
features, tfidf = nlp_utils.apply_tfidf(train_df[text_field])
tfidf_df = pd.DataFrame(features.todense())
if tfidf_df.shape[0] != train_df.shape[0]:
    print("Mismatch on training data sizes")
    print("TFIDF Shape {}".format(tfidf_df.shape))
    print("Train Shape {}".format(train_df.shape))
else:
    features_df = pd.merge(tfidf_df, train_df.reset_index(drop=True), left_index=True, right_index=True).copy()  
    features_df.columns = features_df.columns.astype(str)
#     prediction_utils.async_upload(auth, features_df, "nlp_testcase", "spamtexts")

In [92]:
result = data_management_engine.get_data(auth, "nlp_testcase", "texts", {}, 10000, {}, 0)
print(len(result))

get /getMongoDBFind?database=nlp_testcase&collection=spamtexts&field={}&limit=10000&projections={}&skip=0&
4191


In [ ]:
export_store = "texts"
filetype = "csv"
db = "nlp_testcase"
col = "texts"
projection = "target, text"
data_management_engine.export_documents(auth, export_store, filetype, db, col, "{}", "{}", projection, 0)  

In [ ]:
prediction_utils.frame_csv_file(auth, "texts", "auto", "Quick description")

In [191]:
user_frame = prediction_engine.get_uframe(auth, "autotexts")
print(user_frame)

In [ ]:
result = worker_h2o.prediction_frames(auth)
print(result)

In [ ]:
parms = { 
    "predict_id": "texts_test",
    "description": "This is a test model",
    "model_id": "texts_test_model",
    "model_type": "AUTOML",
    "frame_name": "texts.hex",
    "model_purpose": "NLP",
    "version": "0.0.1",
    "model_parms": {
      "algo": "H2O-AUTOML",
      "training_frame": "texts.hex",
      "validation_frame": "texts.hex",
      "max_models": 10,
      "stopping_tolerance": 0.001,
      "note_stop": "stopping_tolerance of 0.001 for 1m rows and 0.004 for 100k rows",
      "max_runtime_secs": 0,
      "stopping_rounds": 10,
      "stopping_metric": "AUTO",
      "nfolds": 0,
      "note_folds": "nfolds=0 will disable the stacked ensemble creation process",
      "response_column": "pollution",
      "hidden": [
        "1"
      ],
      "exclude_algos": [
        "GLM"
      ]
    }
}

model = worker_h2o.train_model(auth, "spamtexts_test_model", "automl", json.dumps(parms["model_parms"]))
print(model)
print(model.content)